In [27]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score, matthews_corrcoef, average_precision_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint, uniform


def generate_fingerprints(smiles_list):
    fps = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, arr)
        fps.append(arr)
    return np.array(fps)

def evaluate_classifier(true_labels, predictions, probs):
    auc = roc_auc_score(true_labels, probs)
    mcc = matthews_corrcoef(true_labels, predictions)
    avg_precision = average_precision_score(true_labels, probs)
    tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
    spe = tn / (tn + fp)
    sen = tp / (tp + fn)
    ba = (spe + sen)/2
    return {'Held_out_TP': tp, 'Held_out_TN': tn,
            'Held_out_FP': fp, 'Held_out_FN': fn,
            'Held_out_BA': ba,
            'Held_out_AUC': auc, 'Held_out_MCC': mcc, 
            'Held_out_AUCPR': avg_precision, 'Held_out_Specificity': spe,
            'Held_out_Sensitivity': sen}

def fold_error(true_values, predictions):
    ratio = predictions / true_values
    adjusted_ratio = np.where(ratio < 1, 1/ratio, ratio)
    return adjusted_ratio

def evaluate_regression(true_values, predictions):
    rmse = np.sqrt(mean_squared_error(true_values, predictions))
    r2 = np.corrcoef(true_values, predictions)[0, 1] ** 2
    ratio = predictions / true_values
    avg_fold_error = np.mean(fold_error(true_values, predictions))

    return {'Held_out_R2': r2, 'Held_out_RMSE': rmse, "Held_out_average_fold_error": avg_fold_error}

def optimize_threshold_j_statistic(y_true, y_probs):
    # Example usage:
    # y_true is the true labels (binary)
    # y_probs is the predicted probabilities
    # best_threshold = optimize_threshold_j_statistic(y_true, y_probs)

    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    
    # Calculate J statistic values
    j_statistic = tpr - fpr
    
    # Find the index of the threshold that maximizes J statistic
    best_threshold_idx = j_statistic.argmax()
    
    # Get the best threshold
    best_threshold = thresholds[best_threshold_idx]
    
    return best_threshold

# Path where your data is stored
data_path = '../data/processed_splits/'

results = {}

# Assuming PK dataset is regression and others are classification
for dataset in os.listdir(data_path):
    print(dataset)

    # Get all the file names for this dataset
    all_files = os.listdir(os.path.join(data_path, dataset))

    # Extract activity names by removing the _train.csv.gz or _test.csv.gz from file names
    activity_names = list(set([f.replace("_train.csv.gz", "").replace("_test.csv.gz", "") for f in all_files]))

    for activity in tqdm(activity_names, desc="Processing activities"):
        
        train_path = os.path.join(data_path, dataset, f"{activity}_train.csv.gz")
        test_path = os.path.join(data_path, dataset, f"{activity}_test.csv.gz")

        train_df = pd.read_csv(train_path, compression='gzip')
        test_df = pd.read_csv(test_path, compression='gzip')

        X_train = generate_fingerprints(train_df['Standardized_SMILES'])
        X_test = generate_fingerprints(test_df['Standardized_SMILES'])
        y_train = train_df[activity]
        y_test = test_df[activity]

        if dataset == "PK_Lombardo":
            # Regression
            model = RandomForestRegressor(n_jobs=-1)
            model.fit(X_train, y_train)
            predictions_train = model.predict(X_train)
            predictions_test = model.predict(X_test)

            cv_scores = cross_val_score(model, X_train, y_train, n_jobs=20, cv=5, scoring='r2')

            results[activity] = {
                'CV_R2_mean': np.mean(cv_scores),
                'CV_R2_std': np.std(cv_scores),
                **evaluate_regression(y_test, predictions_test)
            }
        else:
            # Classification
            model = RandomForestClassifier(n_jobs=40)
            
            # Hyperparameter Optimization
            param_dist_classification = {'max_depth': randint(10, 20),
                          'max_features': randint(40, 50),
                          'min_samples_leaf': randint(5, 15),
                          'min_samples_split': randint(5, 15),
                          'n_estimators':[200, 300, 400, 500, 600],
                          'bootstrap': [True, False],
                          'oob_score': [False],
                          'random_state': [42],
                          'criterion': ['gini', 'entropy'],
                          'n_jobs': [40],
                          'class_weight' : [None, 'balanced']
                         }
            classification_search = HalvingRandomSearchCV(
                model,
                param_dist_classification,
                factor=3,
                cv=5,
                random_state=42,
                verbose=1,
                n_jobs=40,)
            
            classification_search.fit(X_train, y_train)
            best_model = classification_search.best_estimator_
            
            # Random Over-sampling and Threshold Optimization
            sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
            
            pipeline = Pipeline(steps=[('sampler', sampler), ('model', best_model)])
            pipeline.fit(X_train, y_train)
            
            # Predict using threshold-optimized model
            predictions_train = pipeline.predict(X_train)
            probs_train = pipeline.predict_proba(X_train)[:, 1]
            probs_test = pipeline.predict_proba(X_test)[:, 1]
            
            # Use the optimize_threshold_j_statistic function to find the best threshold
            best_threshold = optimize_threshold_j_statistic(y_train, probs_train)
            #Apply the best threshold to get binary predictions on the test data
            predictions_test = (probs_test >= best_threshold).astype(int)
            
            # Calculate CV AUC using threshold-optimized model
            cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, n_jobs=-1, scoring='roc_auc')

            results[activity] = {
                'CV_AUC_mean': np.mean(cv_scores),
                'CV_AUC_std': np.std(cv_scores),
                **evaluate_classifier(y_test, predictions_test, probs_test)
            }
            
        # Save results at each step
        pd.DataFrame(results).T.to_csv('./structural_model_results.csv')
            
        break

# Save results
results_df = pd.DataFrame(results).T.reset_index(drop=False)
results_df = results_df.rename(columns={'index': 'endpoint'})
results_df.to_csv('./structural_model_results.csv', index=False)

toxcast


Processing activities:   0%|                            | 0/331 [00:00<?, ?it/s]

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 567
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 28
n_resources: 20
Fitting 5 folds for each of 28 candidates, totalling 140 fits
----------
iter: 1
n_candidates: 10
n_resources: 60
Fitting 5 folds for each of 10 candidates, totalling 50 fits
----------
iter: 2
n_candidates: 4
n_resources: 180
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 540
Fitting 5 folds for each of 2 candidates, totalling 10 fits


Processing activities:   0%|                            | 0/331 [00:09<?, ?it/s]


BBBP


...

Processing activities:   0%|                              | 0/5 [00:02<?, ?it/s]


clintox


Processing activities:   0%|                              | 0/1 [00:00<?, ?it/s]

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 848
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 42
n_resources: 20
Fitting 5 folds for each of 42 candidates, totalling 210 fits
----------
iter: 1
n_candidates: 14
n_resources: 60
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 180
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 540
Fitting 5 folds for each of 2 candidates, totalling 10 fits


Processing activities:   0%|                              | 0/1 [00:08<?, ?it/s]


In [28]:
df = pd.read_csv("structural_model_results.csv")
df

,endpoint,CV_AUC_mean,CV_AUC_std,Held_out_TP,Held_out_TN,Held_out_FP,Held_out_FN,Held_out_BA,Held_out_AUC,Held_out_MCC,Held_out_AUCPR,Held_out_Specificity,Held_out_Sensitivity,CV_R2_mean,CV_R2_std,Held_out_R2,Held_out_RMSE,Held_out_average_fold_error
0,ATG_C_EBP_CIS_up,0.744297,0.025942,4.0,110.0,19.0,9.0,0.580203,0.651163,0.125559,0.163976,0.852713,0.307692,NaN,NaN,NaN,NaN,NaN
1,p_np,0.838182,0.017717,67.0,39.0,11.0,17.0,0.788810,0.842857,0.566402,0.891461,0.780000,0.797619,NaN,NaN,NaN,NaN,NaN
2,Investigations,0.681162,0.058890,135.0,12.0,23.0,40.0,0.557143,0.562449,0.098677,0.869581,0.342857,0.771429,NaN,NaN,NaN,NaN,NaN
3,NR-AhR,0.840747,0.053719,37.0,309.0,44.0,12.0,0.815228,0.893623,0.514216,0.651230,0.875354,0.755102,NaN,NaN,NaN,NaN,NaN
4,HIV_active,0.738914,0.129720,0.0,167.0,7.0,7.0,0.479885,0.314450,-0.040230,0.031455,0.959770,0.000000,NaN,NaN,NaN,NaN,NaN
5,DILIst Classification,0.611609,0.011415,84.0,26.0,39.0,29.0,0.571681,0.645882,0.149382,0.766800,0.400000,0.743363,NaN,NaN,NaN,NaN,NaN
6,fraction_unbound_in_plasma_fu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.285169,0.062366,0.474898,0.253559,11.452735
7,CT_TOX,0.694061,0.090749,8.0,179.0,15.0,10.0,0.683562,0.745704,0.329046,0.364659,0.922680,0.444444,NaN,NaN,NaN,NaN,NaN
